In [1]:
#****************************************
# This juptyter notebook was created for the project, Deep Learning Grid Search for
# Diabetes Onset Prediction
# from the Udemy course, Aplied Machine Learning in Healthcare, 
#     https://www.udemy.com/course/applied-machine-learning-for-healthcare
#In this project we complete the following activities:
# 1. FILL THIS IN
#
#*****************************************************************
import sys
import pandas #to handle data sets
import numpy #for computation and arrays
import sklearn #for machine learning packages
import keras
import theano
#theano.test()

#print (sys.path)
print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))
print('Theano: {}'.format(theano.__version__))


Using Theano backend.


Python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Pandas: 0.25.1
Numpy: 1.16.5
Sklearn: 0.21.3
Keras: 2.2.4
Theano: 1.0.4


In [2]:
# we'll be using these library's a lot, so lets make it easier to call them
import pandas as pd
import numpy as np

#import the uci pima indians diabetes dataset
# url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
# since the data set is no longer available in UCI, I found it from another location in github and downloaded it 
#to a file called  'diabetes.doata on my local machine in my machine learning project folder
url = "diabetes.data"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 
         'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url, names = names)

# describe the dataset
df.describe()
         

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [3]:
#Lets look closer at the glucose concentration of 0
df[df['glucose_concentration']==0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [4]:
# Preprocess the data, mark the zero values as NoN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 
         'BMI']  # these are the columns for which a value of 0 doesn't make sense - i.e. is invalid
for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()


,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
# Drop rows with missing values
df.dropna(inplace=True)
# Summarize the number of rows and columns in the df
df.describe()



,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset)
print(dataset.shape)

[[  1.     89.     66.    ...   0.167  21.      0.   ]
 [  0.    137.     40.    ...   2.288  33.      1.   ]
 [  3.     78.     50.    ...   0.248  26.      1.   ]
 ...
 [  2.     88.     58.    ...   0.766  22.      0.   ]
 [ 10.    101.     76.    ...   0.171  63.      0.   ]
 [  5.    121.     72.    ...   0.245  30.      0.   ]]
(392, 9)


In [7]:
# split data set into input (x) and output (Y)
X = dataset[:,0:8]   #take every single row, columns 0-7 - i.e. columns 0 up to but not including column 8
Y = dataset[:,8].astype(int) #take all rows and only the 9th column, convert to integer
print('Shape of X:' + str(X.shape))
print('Shape of Y:' + str(Y.shape))
print('First 5 rows of X')
print(X[:5])
print('First 5 rows of Y')  
print(Y[:5])


Shape of X:(392, 8)
Shape of Y:(392,)
First 5 rows of X
[[1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]
 [3.000e+00 7.800e+01 5.000e+01 3.200e+01 8.800e+01 3.100e+01 2.480e-01
  2.600e+01]
 [2.000e+00 1.970e+02 7.000e+01 4.500e+01 5.430e+02 3.050e+01 1.580e-01
  5.300e+01]
 [1.000e+00 1.890e+02 6.000e+01 2.300e+01 8.460e+02 3.010e+01 3.980e-01
  5.900e+01]]
First 5 rows of Y
[0 1 1 1 1]


In [8]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)  # Fit scaler to our X data
# above - create the matrix that will do the scaling
print('scaler = ' + str(scaler))
# Transform and display the training data
X_standardized = scaler.transform(X)
data = pd.DataFrame(X_standardized)
print('Standardized Data:')
data.describe()

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Standardized Data:


,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [9]:
# import necessary packages
from sklearn.model_selection import GridSearchCV, KFold
# above - Kfod cross validation
from keras.models import Sequential # doing a sequential model
from keras.layers import Dense #Dense layer (fully connected)
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.optimizers import Adam


In [10]:
# start defining the model
def create_model():
    #create model
    model = Sequential() #we'll build it 1 layer at a time
    # 1st layer of a neural network is always the input, so the line below is for the 2nd layer of hte model
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    # above, 8 nodes in the 1st layer
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    # above, 4 notes in the 2nd hidden layer
    model.add(Dense(1, activation='sigmoid'))
    # above, output layer, 1 note. sigmoid pushes output to be a 0 or a 1 which is useful for a binary classification
    
    # compile the model
    adam = Adam(lr = 0.01) # learning rate = 0.1, how fast you want to update parameters in the model
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())
              
              
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# Define a random seed
seed = 6
np.random.seed(seed)

# start defining the model
def create_model():
    #create model
    model = Sequential() #we'll build it 1 layer at a time
    # 1st layer of a neural network is always the input, so the line below is for the 2nd layer of hte model
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    # above, 8 nodes in the 1st layer
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    # above, 4 notes in the 2nd hidden layer
    model.add(Dense(1, activation='sigmoid'))
    # above, output layer, 1 note. sigmoid pushes output to be a 0 or a 1 which is useful for a binary classification
    
    # compile the model
    adam = Adam(lr = 0.01) # learning rate = 0.1, how fast you want to update parameters in the model
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1) #verbose=0 means no printing
 
#define the grid search parameters
batch_size = [10, 20, 40] 
epochs = [10, 50, 100]

#make a dictionary of the grid search paramters
param_grid = dict(batch_size=batch_size, epochs=epochs )
# above batch_size is the key and the values are 10, 20, 40
# above epochs is the key and the value are 10, 50, 100

# build and fit the GridSeachCV (this is where we will do our hyperparamter optimizaion)
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)
print('completed grid.fit')

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))
    


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
261/261 [==============================] - 0s 42us/step - loss: 0.6333 - acc: 0.6897
Epoch 2/10
261/261 [==============================] - 0s 38us/step - loss: 0.4840 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 54us/step - loss: 0.4422 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 54us/step - loss: 0.4246 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 57us/step - loss: 0.4142 - acc: 0.8123
Epoch 6/10
261/261 [==============================] - 0s 50us/step - loss: 0.4022 - acc: 0.8276
Epoch 7/10
261/261 [==============================] - 0s 46us/step - loss: 0.3971 - acc: 0.8352
Epoch 8/10
261/261 [==============================] - 0s 42us/step - loss: 0.3850 - acc: 0.8276
Epoch 9/10
261/261 [==============================] - 0s 54us/step - loss: 0.3844 - acc: 0.8199
Epoch 10/10
131/131 [==============================] - 0s 15us/step
[CV] ............ batch_size=10, epochs=10, score=0.725, total=   1.

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 0s 42us/step - loss: 0.6607 - acc: 0.7280
Epoch 2/10
261/261 [==============================] - 0s 42us/step - loss: 0.6058 - acc: 0.7816
Epoch 3/10
261/261 [==============================] - 0s 46us/step - loss: 0.5757 - acc: 0.7854
Epoch 4/10
261/261 [==============================] - 0s 46us/step - loss: 0.5311 - acc: 0.8123
Epoch 5/10
261/261 [==============================] - 0s 50us/step - loss: 0.5161 - acc: 0.8008
Epoch 6/10
261/261 [==============================] - 0s 46us/step - loss: 0.5176 - acc: 0.7969
Epoch 7/10
261/261 [==============================] - 0s 50us/step - loss: 0.5040 - acc: 0.7969
Epoch 8/10
261/261 [==============================] - 0s 46us/step - loss: 0.4814 - acc: 0.7931
Epoch 9/10
261/261 [==============================] - 0s 50us/step - loss: 0.4738 - acc: 0.8008
Epoch 10/10
131/131 [==============================] - 0s 15us/step
[CV] ............ batch_size=10, epochs=10, score=0.771, total=   1.

...

261/261 [==============================] - 0s 15us/step - loss: 0.3286 - acc: 0.8621
Epoch 47/100
261/261 [==============================] - 0s 27us/step - loss: 0.3328 - acc: 0.8506
Epoch 48/100
261/261 [==============================] - 0s 15us/step - loss: 0.3312 - acc: 0.8659
Epoch 49/100
261/261 [==============================] - 0s 15us/step - loss: 0.3226 - acc: 0.8812
Epoch 50/100
261/261 [==============================] - 0s 19us/step - loss: 0.3209 - acc: 0.8659
Epoch 51/100
261/261 [==============================] - 0s 15us/step - loss: 0.3226 - acc: 0.8621
Epoch 52/100
261/261 [==============================] - 0s 23us/step - loss: 0.3179 - acc: 0.8582
Epoch 53/100
261/261 [==============================] - 0s 15us/step - loss: 0.3168 - acc: 0.8621
Epoch 54/100
261/261 [==============================] - 0s 19us/step - loss: 0.3186 - acc: 0.8544
Epoch 55/100
261/261 [==============================] - 0s 23us/step - loss: 0.3144 - acc: 0.8506
Epoch 56/100
261/261 [===========

262/262 [==============================] - 0s 15us/step - loss: 0.4161 - acc: 0.7863
Epoch 28/100
262/262 [==============================] - 0s 19us/step - loss: 0.4149 - acc: 0.7786
Epoch 29/100
262/262 [==============================] - 0s 19us/step - loss: 0.4143 - acc: 0.7748
Epoch 30/100
262/262 [==============================] - 0s 15us/step - loss: 0.4122 - acc: 0.7748
Epoch 31/100
262/262 [==============================] - 0s 27us/step - loss: 0.4114 - acc: 0.7824
Epoch 32/100
262/262 [==============================] - 0s 15us/step - loss: 0.4089 - acc: 0.7824
Epoch 33/100
262/262 [==============================] - 0s 19us/step - loss: 0.4075 - acc: 0.7863
Epoch 34/100
262/262 [==============================] - 0s 15us/step - loss: 0.4069 - acc: 0.7939
Epoch 35/100
262/262 [==============================] - 0s 15us/step - loss: 0.4028 - acc: 0.7901
Epoch 36/100
262/262 [==============================] - 0s 23us/step - loss: 0.4043 - acc: 0.7939
Epoch 37/100
262/262 [===========

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   44.3s finished
C:\Users\mboucher\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/10
392/392 [==============================] - 0s 20us/step - loss: 0.6724 - acc: 0.6480
Epoch 2/10
392/392 [==============================] - 0s 26us/step - loss: 0.5814 - acc: 0.6684
Epoch 3/10
392/392 [==============================] - 0s 20us/step - loss: 0.5102 - acc: 0.6684
Epoch 4/10
392/392 [==============================] - 0s 23us/step - loss: 0.4888 - acc: 0.6684
Epoch 5/10
392/392 [==============================] - 0s 20us/step - loss: 0.4735 - acc: 0.6684
Epoch 6/10
392/392 [==============================] - 0s 23us/step - loss: 0.4651 - acc: 0.7628
Epoch 7/10
392/392 [==============================] - 0s 23us/step - loss: 0.4599 - acc: 0.7985
Epoch 8/10
392/392 [==============================] - 0s 20us/step - loss: 0.4525 - acc: 0.7959
Epoch 9/10
392/392 [==============================] - 0s 23us/step - loss: 0.4458 - acc: 0.7985
Epoch 10/10
392/392 [==============================] - 0s 20us/step - loss: 0.4393 - acc: 0.7985
completed grid.fit


AttributeError: 'GridSearchCV' object has no attribute 'best_params'